# 필요한 패키지 설치

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import os

# Data불러오기

In [2]:
col_names = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_csv('../Data/MovieLens/ml-100k/u.data', sep='\t', names=col_names)
train_df = pd.read_csv('../Data/MovieLens/ml-100k/ua.base', sep='\t', names=col_names)
test_df = pd.read_csv('../Data/MovieLens/ml-100k/ua.test', sep='\t', names=col_names)

In [3]:
n_users = len(df.loc[:,'user_id'].unique())
n_items = len(df.loc[:,'movie_id'].unique())

# 투입할 Dataset 만들기

In [4]:
# R matrix
R = torch.zeros((n_users, n_items))
for user_id, movie_id, rating, timestamp in train_df.values:
    R[user_id-1, movie_id-1] = rating

In [5]:
R_test = torch.zeros((n_users, n_items))
for user_id, movie_id, rating, timestamp in test_df.values:
    R_test[user_id-1, movie_id-1] = rating

In [6]:
class PandasDataset(Dataset):
    
    def __init__(self, dataset): 
        super(PandasDataset, self).__init__()
        self.X = dataset.iloc[ : , [0,1]]
        self.y = dataset.iloc[ :, 2]
        self.X_value, self.y_value = self.X.values, self.y.values
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        return{
            'X' : torch.from_numpy(self.X_value)[idx],
            'y' : torch.from_numpy(self.y_value)[idx]
        }
        

In [7]:
batch_size = 1000

train_dataset = PandasDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Device

In [8]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.is_available()

True

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Model

In [10]:
# number of latent factor
k = 10

In [11]:
class SVD(nn.Module):
    def __init__(self):
        super(SVD, self).__init__()
        
        self.mu = nn.Parameter(torch.tensor(np.mean(train_df.loc[:, 'rating'])), requires_grad = True) # scalar
        
        self.B_u = nn.Parameter(torch.randn(n_users), requires_grad = True) # 1 x n_users
        self.B_i = nn.Parameter(torch.randn(n_items), requires_grad = True) # 1 x n_items
        
        self.P = nn.Parameter(nn.init.normal_(torch.zeros(k, n_users), std=1.0/k), requires_grad = True) # k x n_users
        self.Q = nn.Parameter(nn.init.normal_(torch.zeros(k, n_items), std=1.0/k), requires_grad = True) # k x n_items
        
    def forward(self):
        
        user_bias = self.B_u.repeat(n_items).view(n_users,n_items) # n_users x n_items
        item_bias = self.B_i.repeat(n_users).view(n_users,n_items) # n_users x n_items
        
        output = torch.mm(self.P.T, self.Q) + user_bias + item_bias # n_users x n_items
        
        reg = torch.norm(self.P) + torch.norm(self.Q) + torch.norm(self.B_u) + torch.norm(self.B_i) # scalar
        
        return output, reg
    

In [12]:
model = SVD()
model = model.to(device)

# Learning

In [13]:
epochs = 300
lamda_3 = 0.005
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [14]:
writer = SummaryWriter()

for epoch in range(epochs):
    total_loss = 0
    model.train()
    for data in train_loader: # batch x 2
        
        # predict
        R_hat = model()[0]
        reg = model()[1]
        
        # loss
        loss = torch.norm(R[R!=0].to(device)-R_hat[R!=0]) + lamda_3*reg
        
        # initialize
        optimizer.zero_grad()
        
        # calculate gradient
        loss.backward()
        
        # update
        optimizer.step()
        
        total_loss += loss.item()
        
    obj = total_loss / len(train_loader)   
    
    #evaluation
    model.eval()
    
    R_hat = model()[0]
    SSE = torch.norm(R_test[R_test!=0].to(device) - R_hat[R_test!=0])
    RMSE = torch.sqrt(SSE/ R_test.shape[0])
    
    writer.add_scalar("obj", obj, epoch+1)
    writer.add_scalar("RMSE", RMSE, epoch+1)
    
    if (epoch+1) % 10 == 0:
        print("epoch : {}, obj : {}, RMSE : {}".format(epoch+1, obj, RMSE))

epoch : 10, obj : 1018.1600214360834, RMSE : 0.5898470282554626
epoch : 20, obj : 845.935043167282, RMSE : 0.5359881520271301
epoch : 30, obj : 701.4034725650326, RMSE : 0.4875205457210541
epoch : 40, obj : 586.98974609375, RMSE : 0.4465717673301697
epoch : 50, obj : 501.9519505762792, RMSE : 0.41429397463798523
epoch : 60, obj : 441.98416774875517, RMSE : 0.3903130292892456
epoch : 70, obj : 400.9330075442136, RMSE : 0.3731512427330017
epoch : 80, obj : 373.0351106413118, RMSE : 0.36108335852622986
epoch : 90, obj : 353.94774250407795, RMSE : 0.3526441156864166
epoch : 100, obj : 340.6921071482229, RMSE : 0.3467302918434143
epoch : 110, obj : 331.27991862873455, RMSE : 0.34254834055900574
epoch : 120, obj : 324.3895293854095, RMSE : 0.3395415246486664
epoch : 130, obj : 319.1409385597313, RMSE : 0.3373263478279114
epoch : 140, obj : 314.944083748283, RMSE : 0.3356415927410126
epoch : 150, obj : 311.39733048323745, RMSE : 0.3343091905117035
epoch : 160, obj : 308.22185575045074, RMSE :